In [ ]:
import pandas as pd
import numpy as np
import math
from scipy.stats import norm
from statsmodels.sandbox.stats.multicomp import multipletests
from collections import OrderedDict

In [ ]:
path='../Result_dup/'
ctrl1=pd.read_csv(path+'ctrl1_fitting_aggr.csv',low_memory=False)
ctrl2=pd.read_csv(path+'ctrl2_fitting_aggr.csv',low_memory=False)
treated1=pd.read_csv(path+'treated1_fitting_aggr.csv',low_memory=False)
treated2=pd.read_csv(path+'treated2_fitting_aggr.csv',low_memory=False)

Pgroups=pd.read_csv(path+'Pgroups.csv')

ctrl1.loc[ctrl1[(ctrl1['ctrl1_R']<0.8) | (ctrl1['ctrl1_Pl']>0.3)].index,ctrl1.columns[4:]]=float('nan')
ctrl2.loc[ctrl2[(ctrl2['ctrl2_R']<0.8) | (ctrl2['ctrl2_Pl']>0.3)].index,ctrl2.columns[4:]]=float('nan')

treated1.loc[treated1[treated1['treated1_R']<0.8].index,treated1.columns[4:]]=float('nan')
treated2.loc[treated2[treated2['treated2_R']<0.8].index,treated2.columns[4:]]=float('nan')


In [ ]:
diff_rep1 = pd.DataFrame(index=range(0,ctrl1.shape[0]))
diff_rep1['diff_rep1'] = treated1['treated1_min']-ctrl1['ctrl1_min']
diff_rep1['slope_rep1'] = np.fmin(ctrl1['ctrl1_slope'],treated1['treated1_slope'])
diff_rep2 = pd.DataFrame(index=range(0,ctrl2.shape[0]))
diff_rep2['diff_rep2'] = treated2['treated2_min']-ctrl2['ctrl2_min']
diff_rep2['slope_rep2'] = np.fmin(ctrl2['ctrl2_slope'],treated2['treated2_slope'])

In [ ]:
def correct_fdr(p_values):
   p_values= {k:v for k,v in enumerate(p_values)}
   n_comparison= len(p_values)
   modifier= n_comparison

   indexed_p_values= OrderedDict({(j[1],i):j[0] for i, j in enumerate(p_values.items())})

   indexed_p_values= OrderedDict(sorted(indexed_p_values.items(), reverse=True))

   for (value, i),ind in indexed_p_values.items():
       new_value= p_values[ind] * n_comparison / float(modifier)

       p_values[ind]= new_value if new_value <=1 else 1 
       modifier -= 1 

   fdr= [1]*len(p_values)

   for i,value in p_values.iteritems():
       fdr[i]= value 

   return fdr
def combine_col(df,string):
    df1=df[df.columns[0:13]]
    df1['mp_'+string]=df[string+'_min'].tolist()
    df1[string+'_slope']=df[string+'_slope'].tolist()
    return df1
def diff_rep(df1,string,groupname):
    use_me=list(df1['slope_' + string].dropna().index)
    slope_rep1=df1.ix[use_me]['slope_' + string].sort_values(ascending=False)
    slope_order=slope_rep1.index
    diff_re=df1.ix[slope_order]['diff_' + string]
    diff_quant = []
    diff_z = [float('nan')]*len(diff_re)
    bin_size=300
    for i in range(0,int(np.floor(len(diff_re)/bin_size))):
        if len(diff_re) - bin_size*(i+1) < bin_size:
            y = len(diff_re)
        else:
            y = bin_size * (i+1)
        l = diff_re.iloc[(bin_size*(i)): y].quantile([.1587, .50, .8413])
        for j in range((bin_size*(i)),y):
            if j == 1267:
                print('haha')
            if (np.isnan(diff_re.iloc[j])):
                diff_z[j] =float('nan')
            elif (diff_re.iloc[j] < l.ix[.50]):
                diff_z[j] = (l.ix[.50] - diff_re.iloc[j])/(l.ix[.50]-l.ix[.1587])
            else:
                diff_z[j] = (diff_re.iloc[j]-l.ix[.50])/(l.ix[.8413]-l.ix[.50])
        diff_quant.append(l.tolist())
    diff_z_abs=map(abs,diff_z)
    diff_prob=1-norm.cdf(np.abs(diff_z))
    diff_prob1=[i for i in range(0,len(diff_prob)) if not(np.isnan(diff_prob[i]))]
    diff_prob2=[diff_prob[i] for i in diff_prob1]
    diff_prob2=multipletests(diff_prob2,method='fdr_bh')[1]
    for i in range(0,len(diff_prob1)):
        diff_prob[diff_prob1[i]]=diff_prob2[i]
    groupname=groupname.reset_index(drop=True)
    names=groupname.ix[use_me]
    df2 = pd.concat([names.ix[slope_order],df1.ix[slope_order]['diff_'+string]],1)
    df2['diff_z_'+string]=diff_z
    df2['diff_prob_'+string]=diff_prob
    df2= pd.concat([df2,slope_rep1.ix[slope_order]],1)
    return df2

def diff_repAll(df1,string,groupname):
    use_me=list(df1['slope_' + string].dropna().index)
    slope_rep1=df1.ix[use_me]['slope_' + string].sort_values(ascending=False)
    slope_order=slope_rep1.index
    diff_re=df1.ix[slope_order]['diff_' + string]
    diff_quant = []
    diff_z = [float('nan')]*len(diff_re)
    l = diff_re.quantile([.1587, .50, .8413])
    for j in range(0,len(diff_re)):
        if (np.isnan(diff_re.iloc[j])):
            diff_z[j] =float('nan')
        elif (diff_re.iloc[j] < l.ix[.50]):
            diff_z[j] = (l.ix[.50] - diff_re.iloc[j])/(l.ix[.50]-l.ix[.1587])
        else:
            diff_z[j] = (diff_re.iloc[j]-l.ix[.50])/(l.ix[.8413]-l.ix[.50])
    diff_quant.append(l.tolist())
    diff_z_abs=map(abs,diff_z)
    diff_prob=1-norm.cdf(np.abs(diff_z))
    diff_prob1=[i for i in range(0,len(diff_prob)) if not(np.isnan(diff_prob[i]))]
    diff_prob2=[diff_prob[i] for i in diff_prob1]
    diff_prob2=multipletests(diff_prob2,method='fdr_bh')[1]
    for i in range(0,len(diff_prob1)):
        diff_prob[diff_prob1[i]]=diff_prob2[i]
    groupname=groupname.reset_index(drop=True)
    names=groupname.ix[use_me]
    df2 = pd.concat([names.ix[slope_order],df1.ix[slope_order]['diff_'+string]],1)
    df2['diff_z_'+string]=diff_z
    df2['diff_prob_'+string]=diff_prob
    df2= pd.concat([df2,slope_rep1.ix[slope_order]],1)
    return df2

In [ ]:
diff_rep1=diff_rep(diff_rep1,'rep1',Pgroups)
diff_rep2=diff_rep(diff_rep2,'rep2',Pgroups)


In [ ]:
ctrl1fit=combine_col(ctrl1,'ctrl1')
ctrl2fit=combine_col(ctrl2,'ctrl2')
treated1fit=combine_col(treated1,'treated1')
treated2fit=combine_col(treated2,'treated2')

In [ ]:
result=ctrl1fit.merge(treated1fit,on=["Protein Group Accessions", "Gene_Name", "Protein_Description"]).merge(ctrl2fit,on=["Protein Group Accessions", "Gene_Name", "Protein_Description"])\
.merge(treated2fit,on=["Protein Group Accessions", "Gene_Name", "Protein_Description"]).merge(diff_rep1,on=["Protein Group Accessions", "Gene_Name", "Protein_Description"])\
.merge(diff_rep2,on=["Protein Group Accessions", "Gene_Name", "Protein_Description"])

result.to_csv(path+'peptides_fit_aggr.csv',index=False)

In [ ]:
same_thermal_shift_direction = (result['diff_rep1'] < 0) == (result['diff_rep2'] < 0)
slope_steep_enough = (result['slope_rep1'] < -0.06) | (result['slope_rep2'] < -0.06)
thermal_shift_greater_than_difference_vehicle = np.fmin(map(abs,result['diff_rep1']),map(abs,result['diff_rep2'])) > map(abs,result['mp_ctrl1']-result['mp_ctrl2'])
x = result["diff_prob_rep1"] < 0.1
y = result["diff_prob_rep2"] < 0.1
pvalue_criteria = (x == True) & (y == True)
fulfills_all =  (same_thermal_shift_direction==True)  & (thermal_shift_greater_than_difference_vehicle == True) & (pvalue_criteria==True) & (slope_steep_enough==True)
in_both_replicates = (~np.isnan(result["diff_rep1"])) & (~np.isnan(result["diff_rep2"]))

In [ ]:
criteria=pd.concat([same_thermal_shift_direction, slope_steep_enough, in_both_replicates,pd.Series(thermal_shift_greater_than_difference_vehicle), pvalue_criteria, fulfills_all],1)
criteria=criteria.fillna(False)
criteria.columns=['same_thermal_shift_direction', 'slope_steep_enough', 'in_both_replicates','thermal_shift_greater_than_difference_vehicle', 'pvalue_criteria', 'fulfills_all']
result=pd.concat([result,criteria],1)

In [ ]:
res_temp=['A0A0A0MRV3','Q10567-3','P10412','Q9H501','A0A024R884','Q6AWC2-4','P49257','Q9Y6V0-2','P04181','P15104','Q15836','Q5MNZ9-2','Q9BY42','Q08426','Q9BTV6','P24666','Q14315','P08758','O14907','P06733-2','B4DIQ0','V9HVZ7','Q9NSV4-5','P36639-4','Q99805','P20248','Q8N3F4','P28161','O15400','Q6I9Y2','Q14258','O00330-3','B2RN10','Q9H845','Q562Z4','Q9UGR2-2','B2R5H0','Q8TB61-3','Q96PK6']

In [ ]:
result[(~(result['fulfills_all']==True))&(result['Protein Group Accessions'].isin(res_temp))]

In [ ]:
# result[result['fulfills_all']==True][['Protein Group Accessions','mp_ctrl1','mp_treated1','mp_ctrl2','mp_treated2']]
result[result['fulfills_all']==True].shape

In [ ]:
result.to_csv(path+'result_aggr_end.csv',index=False)